In [ ]:
pip install unsloth transformers trl

In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    load_in_4bit=True
 )

In [ ]:
model = FastLanguageModel.get_peft_model(
    model, r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

In [ ]:
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

In [ ]:
dataset = load_dataset("mlabonne/FineTome-100k", split="train")

dataset = dataset.shuffle(seed=42).select(range(1000))
split_dataset = dataset.train_test_split(test_size=0.1)

train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

In [ ]:
# Apply standardization separately to train and validation
train_dataset = standardize_sharegpt(train_dataset)
val_dataset = standardize_sharegpt(val_dataset)


In [ ]:
dataset

In [ ]:
train_dataset[0]
val_dataset[0]


In [ ]:
train_dataset = train_dataset.map(
    lambda examples: {
        "text": [
            tokenizer.apply_chat_template(convo, tokenize=False)
            for convo in examples["conversations"]
        ]
    },
    batched=True
)

val_dataset = val_dataset.map(
    lambda examples: {
        "text": [
            tokenizer.apply_chat_template(convo, tokenize=False)
            for convo in examples["conversations"]
        ]
    },
    batched=True
)


In [ ]:
train_dataset
val_dataset


In [ ]:
train_dataset[0]
val_dataset[0]


In [ ]:
!pip install evaluate


In [ ]:
!pip install rouge_score


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=2048,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="steps",
        save_steps=50,
        eval_strategy="steps",
        eval_steps=50,
    ),
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("finetuned_model1")

In [ ]:
inference_model, inference_tokenizer = FastLanguageModel.from_pretrained(
    model_name="./finetuned_model1",
    max_seq_length=2048,
    load_in_4bit=True
)

In [ ]:
import torch
import evaluate
from tqdm import tqdm

small_val = val_dataset.select(range(min(50, len(val_dataset))))
rouge_metric = evaluate.load("rouge")

all_preds = []
all_labels = []

print("Calcul des scores ROUGE...")
for example in tqdm(small_val):
    conv = example["conversations"]

    # Swapped 'value' for 'content'
    prompt_text = conv[0]["content"]
    target_text = conv[1]["content"]

    formatted_prompt = inference_tokenizer.apply_chat_template([{
        "role": "user",
        "content": prompt_text
    }], tokenize=False, add_generation_prompt=True)

    model_inputs = inference_tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_ids = inference_model.generate(
            **model_inputs,
            max_new_tokens=512,
            temperature=0.1,
            do_sample=False,
            pad_token_id=inference_tokenizer.pad_token_id
        )

    input_len = model_inputs.input_ids.shape[1]
    response = inference_tokenizer.decode(generated_ids[0][input_len:], skip_special_tokens=True)

    all_preds.append(response)
    all_labels.append(target_text)

rouge_results = rouge_metric.compute(predictions=all_preds, references=all_labels)

print("\n--- Scores ROUGE Professionnels ---")
for key, value in rouge_results.items():
    print(f"{key.upper()}: {value*100:.2f}%")

In [ ]:
text_prompts = [
    "What are the key principles of investment?"
]

for prompt in text_prompts:
    formatted_prompt = inference_tokenizer.apply_chat_template([{
        "role": "user",
        "content": prompt
    }], tokenize=False, add_generation_prompt=True)

    model_inputs = inference_tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    generated_ids = inference_model.generate(
        **model_inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        pad_token_id=inference_tokenizer.pad_token_id
    )

    response = inference_tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[-1]:], skip_special_tokens=True)

    print(f"Prompt: {prompt}\n")
    print(f"Assistant: {response}\n")
    print("-" * 50)